In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import operator
from sklearn import preprocessing
import seaborn as sb
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("futbol.csv", parse_dates= True, index_col= 1)

In [3]:
dataset = np.array(df.filter(["Nationality","Club","Overall","Potential","Value","Wage"]).values)

In [4]:
unique, counts = np.unique(dataset[:,0], return_counts=True)

In [5]:
newDataset = dataset

In [7]:
for i in range(np.where(counts<dataset.shape[0]/unique.shape[0])[0].shape[0]):
    newDataset=np.delete(newDataset,np.where(newDataset == np.delete(unique,np.where(counts>dataset.shape[0]/unique.shape[0]))[i])[0],axis=0)

In [ ]:
unique, counts = np.unique(newDataset[:,0], return_counts=True)

In [ ]:
promedios = [sum([dataset[j][2] for j in np.where(dataset == unique[i])[0]])/np.where(dataset == unique[i])[0].shape[0] for i in range(len(unique))]

In [ ]:
dic = {x[0]:x[1] for x in zip(unique,promedios)}
teams_sort = dict(sorted(dic.items(), key=operator.itemgetter(1), reverse=True))

In [ ]:
plt.figure(figsize=(30,12))
plt.bar(teams_sort.keys(),teams_sort.values())
plt.ylim(55,75)
plt.yticks(fontsize=24)
plt.xticks(list(teams_sort.keys()), rotation='vertical', fontsize=28)
plt.grid()
plt.show()

In [ ]:
borrar = "€K"
for i in range(0,2,1):
    for j in range(len(newDataset[:,4+i])):
        for k in range(len(borrar)):
            newDataset[:,4+i][j] = newDataset[:,4+i][j].replace(borrar[k],"")
for i in range(0,2,1):
    for j in range(len(newDataset[:,4+i])):
        if newDataset[:,4+i][j][-1] == "M":
            newDataset[:,4+i][j] = float(newDataset[:,4+i][j].replace("M",""))*1000

In [ ]:
for n in range(2):
    newDataset[:,4+n] = [float(num) for num in newDataset[:,4+n]]
for n in range(4):
    newDataset[:,2+n] = (newDataset[:,2+n] - min(newDataset[:,2+n]))/(max(newDataset[:,2+n])-min(newDataset[:,2+n]))

In [ ]:
for pais in unique:
    plt.figure(figsize=(15,6))
    plt.plot(newDataset[np.where(newDataset == pais)[0]][:,5])
    plt.plot(newDataset[np.where(newDataset == pais)[0]][:,2])
    plt.grid()
    plt.ylim(0,1)
    plt.title(pais, fontsize=20)
    plt.show()
    print(np.corrcoef(list(newDataset[np.where(newDataset == pais)[0]][:,5]),list(newDataset[np.where(newDataset == pais)[0]][:,2]))[0][1])

In [ ]:
newDf = pd.DataFrame(data=newDataset,columns=["Pais","Club","Overall","Potential","Value","Salario"])
newDf

In [ ]:
sb.pairplot(newDf.dropna(), hue='Pais',size=4,vars=["Overall","Potential","Value","Salario"])

In [ ]:
sb.pairplot(newDf.dropna(), hue='Club',size=4,vars=["Overall","Potential","Value","Salario"])

In [ ]:
newDf["Pais"] = newDf.Pais.astype("category").cat.codes
newDf["Club"] = newDf.Club.astype("category").cat.codes
newDf["Pais"] = (newDf["Pais"] - min(newDf["Pais"]))/ (max(newDf["Pais"])-min(newDf["Pais"]))
newDf["Club"] = (newDf["Club"] - min(newDf["Club"]))/ (max(newDf["Club"])-min(newDf["Club"]))

In [ ]:
newDf

In [ ]:

mCorr=newDf.corr().round(2)
mCorr

In [ ]:
input_train, input_test, target_train, target_test = train_test_split(newDf.drop(["Pais","Club"],axis=1), newDf["Pais"], test_size = 0.3, random_state=4,shuffle=True)

model = Sequential()
model.add(layers.Conv2D())
model.add((100, activation = 'relu', return_sequences = True, input_shape = (step_days ,input_train.shape[2])))
model.add(LSTM(32, activation = 'relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(3))
model.compile(optimizer = 'adam', loss = 'mse')
history = model.fit(input_train, target_train, validation_data = (input_test, target_test),batch_size = 16, epochs = 100)